In [1]:
import twitter
import sys

#language detection
import langid

#package for sentiment analysis
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()

# get the credentials
sys.path.insert(0, '../load')
from config import credentials

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\karar\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
api = twitter.Api(consumer_key = credentials['CONSUMER_KEY'],
                 consumer_secret = credentials['CONSUMER_SECRET'],
                 access_token_key = credentials['ACCESS_TOKEN'],
                 access_token_secret = credentials['ACCESS_SECRET'])

In [9]:
def fetch_twitter_data(sia, api, start, end, coin, size = 50):
    """
    Given sentiment analyzer, a date range and coin:
    1) Get size twitter posts about that coin.
    2) Get sentiment scores.
        restrict_score must be in the format >int or <int
        max value for size is 100
    Date ranges are expected in 'YYY-MM-DD' format
    """
                
    def get_sentiment(posts, sia):
        """Given a set of posts, runs sentiment analysis on the titles and returns the posts with the analysis."""
        for post in posts:
            post['sentiment_scores'] = sia.polarity_scores(post['text'])
        return posts
    
    # convert twitter module to object     
    res = api.GetSearch(term = coin, count = size, since = start, until = end, lang='en', result_type='popular')
    posts = []
    for status in res:
        posts.append({
            'text': status.text,
            'created:': status.created_at,
            'favorites': status.favorite_count,
            'retweets': status.retweet_count,
            'followers': status.user.followers_count
        })
        
    # get sentiment analysis
    posts = get_sentiment(posts, sia)
    return posts
    
    

In [10]:
fetch_twitter_data(sia, api, '2021-10-01', '2021-10-02', 'Ethereum', size = 1)

[{'text': '#AuctionUpdate Curio Cards, a set of 31 NFTs considered to be the oldest artworks on the Ethereum blockchain sold f… https://t.co/VXbjJRxVai',
  'created:': 'Fri Oct 01 14:35:11 +0000 2021',
  'favorites': 1591,
  'retweets': 591,
  'followers': 168018,
  'sentiment_scores': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}}]

In [6]:
help(api.GetSearch)

Help on method GetSearch in module twitter.api:

GetSearch(term=None, raw_query=None, geocode=None, since_id=None, max_id=None, until=None, since=None, count=15, lang=None, locale=None, result_type='mixed', include_entities=None, return_json=False) method of twitter.api.Api instance
    Return twitter search results for a given term. You must specify one
    of term, geocode, or raw_query.
    
    Args:
      term (str, optional):
        Term to search by. Optional if you include geocode.
      raw_query (str, optional):
        A raw query as a string. This should be everything after the "?" in
        the URL (i.e., the query parameters). You are responsible for all
        type checking and ensuring that the query string is properly
        formatted, as it will only be URL-encoded before be passed directly
        to Twitter with no other checks performed. For advanced usage only.
        *This will override any other parameters passed*
      since_id (int, optional):
        Ret